In [2]:
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 18.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter

2025-03-13 10:36:58.434704: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-13 10:36:58.644528: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-13 10:36:58.807362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741858618.945081    1444 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741858618.983362    1444 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 10:36:59.322758: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [30]:
# === Chemins ===
input_dir = os.path.join("wsl.localhost", "Ubuntu", "home","thomas","code","ThomasLechanoine","projet","data","Image_2_Breast_Cancer")
output_dir = os.path.join("wsl.localhost", "Ubuntu","home","thomas","code","ThomasLechanoine","projet","data,processed_images")

# === Création du dossier de sortie avec la même structure ===
for split in ["train", "valid", "test"]:
    for label in ["0", "1"]:
        os.makedirs(os.path.join(output_dir, split, label), exist_ok=True)

ls: cannot access 'UsersASUSDesktopPreproTL': No such file or directory


In [31]:
# === Fonction de prétraitement ===
def preprocess_image(image_path, output_path):
    """Charge une image, la prétraite et l'enregistre sans écraser l'originale."""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))

    # Réduction du bruit
    img = cv2.GaussianBlur(img, (3, 3), 0)

    # Normalisation entre 0 et 1
    img = img.astype("float32") / 255.0

    # Sauvegarde (convertie en 0-255 pour éviter perte de qualité)
    img = (img * 255).astype("uint8")
    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

# === Prétraitement de toutes les images ===
for split in ["train", "valid", "test"]:
    for label in ["0", "1"]:
        input_folder = os.path.join(input_dir, split, label)
        output_folder = os.path.join(output_dir, split, label)

        for filename in os.listdir(input_folder):
            if filename.endswith(".jpg") :
                input_path = os.path.join(input_folder, filename)
                output_path = os.path.join(output_folder, filename)
                preprocess_image(input_path, output_path)

print("✅ Prétraitement terminé !")

FileNotFoundError: [Errno 2] No such file or directory: 'wsl.localhost/Ubuntu/home/thomas/code/ThomasLechanoine/projet/data/Image_2_Breast_Cancer/train/0'

In [19]:
!ls -ld /home/thomas/code/ThomasLechanoine/projet/data/Image_2_Breast_Cancer/train/0

ls: cannot access '/home/thomas/code/ThomasLechanoine/projet/data/Image_2_Breast_Cancer/train/0': No such file or directory


In [ ]:
# === Comptage des images par classe pour équilibrage ===
num_images = {label: len(os.listdir(os.path.join(output_dir, "train", label))) for label in ["0", "1"]}
print(f"📊 Nombre d'images : {num_images}")

# === Définir la classe majoritaire et minoritaire ===
majority_class = "0" if num_images["0"] > num_images["1"] else "1"
minority_class = "1" if majority_class == "0" else "0"

# === Facteur d'augmentation pour équilibrer les classes ===
augmentation_factor = num_images[majority_class] // num_images[minority_class]
print(f"📈 Augmentation de la classe {minority_class} x{augmentation_factor}")

# === Data Augmentation uniquement pour la classe minoritaire ===
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

minority_folder = os.path.join(output_dir, "train", minority_class)
for filename in os.listdir(minority_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(minority_folder, filename)
        img = tf.keras.utils.load_img(img_path)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        # Générer plusieurs augmentations
        aug_iter = datagen.flow(img_array, batch_size=1)
        for i in range(augmentation_factor - 1):  # -1 car on a déjà l'image originale
            aug_img = next(aug_iter)[0].astype("uint8")
            aug_filename = f"aug_{i}_{filename}"
            aug_img_path = os.path.join(minority_folder, aug_filename)
            tf.keras.preprocessing.image.save_img(aug_img_path, aug_img)

print("✅ Augmentation terminée !")

In [ ]:
# === Chargement des images avec tf.data ===
def load_and_preprocess(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0  # Normalisation
    return img, label

def get_dataset(split):
    images = []
    labels = []
    v
    for label in ["0", "1"]:
        folder = os.path.join(output_dir, split, label)
        for filename in os.listdir(folder):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                images.append(os.path.join(folder, filename))
                labels.append(int(label))

    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.shuffle(len(images)).map(load_and_preprocess).batch(32)
    return dataset

# Charger les datasets
train_dataset = get_dataset("train")
valid_dataset = get_dataset("valid")
test_dataset = get_dataset("test")

print("✅ Chargement des datasets terminé !")